In [1]:
from collections import OrderedDict

import numpy as np
import trimesh
import open3d as o3d
import torch
import os
import glob


from src.polar_traversal import gather_all_paths, sample_all_paths

np.set_printoptions(precision=4, suppress=True)
torch.set_printoptions(precision=4, sci_mode=False)

In [2]:
import numpy as np
import trimesh
import open3d as o3d
import torch
import torch.nn.functional as F

import math

from shapely.geometry import LineString, Point

# np.set_printoptions(precision=4, suppress=True)
# torch.set_printoptions(precision=4, sci_mode=False)

def scale_mesh(stl_path, offset=0.02):
    mesh = trimesh.load(stl_path)    
    vertices = torch.from_numpy(mesh.vertices).float()
    vertices = vertices / vertices.abs().max() * (1.-offset)        
    return trimesh.Trimesh(vertices=vertices, faces=mesh.faces)

def flip_mesh(mesh):
    vertices = np.array(mesh.vertices)
    switched = np.vstack([vertices[:, 1], vertices[:, 2],  vertices[:, 0]]).transpose()
    return trimesh.Trimesh(vertices=switched, faces=np.array(mesh.faces))

def get_start(mesh):
    ray_origins = np.array([[0, 0, 2],])
    ray_directions = np.array([[0, 0, -1]])

    locations, _, index_tri = mesh.ray.intersects_location(
            ray_origins=ray_origins,
            ray_directions=ray_directions)
    top_face_idx = locations[:, 2].argmax()
    start_face = index_tri[top_face_idx] 
    start_point =  locations[top_face_idx]
    start_normal = np.array([0, 0, +1])
    assert start_point.shape == start_normal.shape, (start_point, start_normal)
    return start_face, start_point, start_normal

class Bridge:

    def __init__(self, path):
        self.path = path
        self.mesh = flip_mesh(scale_mesh(path))
        self.mesh.fix_normals()
        self.faces = self.make_faces()
        self.polars = math.pi + np.array(
            np.arctan2(self.mesh.vertices[:, 1], 
                       self.mesh.vertices[:, 0]))
        self.edge_to_face = self.make_edge_to_face()

    def make_faces(self):
        faces = np.array(self.mesh.faces)
        faces.sort(axis=1)
        return faces
    
    def make_edge_to_face(self):        
        edge_to_face = {}
        for i, face in enumerate(self.faces):
            edges = self.get_face_edges(face)
            for edge in edges:
                if edge in edge_to_face:
                    edge_to_face[edge].append(i)
                else:
                    edge_to_face[edge] = [i]
        return edge_to_face
    
    def get_face_edges(self, face):
        return [(face[0], face[1]),
                (face[0], face[2]),
                (face[1], face[2]),]
        
    def get_face_ids(self, edge, past_face_ids):        
        if edge in self.edge_to_face:
            face_ids = self.edge_to_face[edge]
            
            res =  [f for f in face_ids if f not in past_face_ids]
            #print('get_face_ids', face_ids, past_face_ids, res)
            return res
        return []    
    
    def next_edges_face_id(self, edge, past_face_ids, past_edges):
        face_ids = self.get_face_ids(edge, past_face_ids)
        face_id, edges = None, []
        if face_ids:
            face_id = face_ids[0] # Pick first face             
            face =  self.faces[face_id]
            face_edges = self.get_face_edges(face)
            for face_edge in face_edges:
                if face_edge not in past_edges:
                    edges.append(face_edge)                
        return face_id, edges
            
        
    def __repr__(self):
        return f'Path: {self.path}'
        
    def scale_mesh(self, stl_path, offset=0.1):
        mesh = trimesh.load(stl_path)    
        vertices = torch.from_numpy(mesh.vertices).float()
        vertices = vertices / vertices.abs().max() * (1.-offset)        
        return trimesh.Trimesh(vertices=vertices, faces=mesh.faces)
    
def get_intersection_point(latitude, edge_vertex, edge_vertex_normals):
    ev_2d  = edge_vertex[:, :2]
    e1_pt, e2_pt = Point(ev_2d[0]),  Point(ev_2d[1])
    
    l1 = LineString(latitude)
    l2 = LineString(ev_2d)
    if l1.intersects(l2):
        pt = l1.intersection(l2)        
        xy = np.array(pt.xy)
        d1 = e1_pt.distance(pt)
        d2 = e1_pt.distance(e2_pt)
        assert d1 <= d2,  ('d1 > d2',  pt, d1, d2)
        ratio = d1 / d2 if d2 != 0 else 0.5
        xyz = ratio * edge_vertex[0] + (1-ratio) *  edge_vertex[1]
        normal =  ratio * edge_vertex_normals[0] + (1-ratio) *  edge_vertex_normals[1]
        point, normal =  np.array(xyz), np.array(normal)
        assert point.shape == normal.shape, (edge_vertex, edge_vertex_normals)
        return point, normal        
    return None, None
   
def it_intersects(edge, latitude, bridge):
    edge_sel = np.array(edge)    
    edge_vertex =  bridge.mesh.vertices[edge_sel]        
    edge_vertex_normals =  bridge.mesh.vertex_normals[edge_sel]            
    return get_intersection_point(latitude, edge_vertex, edge_vertex_normals)
    

def make_latitudes(n):
    r_angle = torch.stack((
        torch.zeros(n) + 2,
        torch.arange(0, n) * (2 * math.pi / n)))
    xy = torch.stack((r_angle[0]*torch.cos(r_angle[1]),
                      r_angle[0]*torch.sin(r_angle[1])))#.t()
    latitudes = torch.stack((torch.zeros_like(xy), xy))
    return latitudes.permute(2, 0, 1).numpy()    

def get_intersection(latitude, edges, bridge):
    #print('get_intersection', edges)
    for edge in edges:
        edge_line = bridge.mesh.vertices[np.array(edge)]                
        point, normal = it_intersects(edge, latitude, bridge)
        if point is not None:            
            return point, normal,  edge    
    return None, None, None
    
def gather_path(latitude, face_id, point, normal, bridge):    
    face = bridge.faces[face_id]
    edges = bridge.get_face_edges(face)
    path = { 'points': [point], 'normals': [normal]}    
    point, normal, edge = get_intersection(latitude, edges, bridge)        
    path['points'].append(point)
    path['normals'].append(normal) 
    past_face_ids, past_edges = [face_id], edges.copy()
    while True:        
        face_id, edges = bridge.next_edges_face_id(edge, past_face_ids, past_edges)
        #print(past_edges)
        if face_id is not None:
            past_face_ids.append(face_id)
            past_edges = past_edges + edges
            point, normal, edge = get_intersection(latitude, edges, bridge)
            if point is not None:                
                path['points'].append(point) 
                path['normals'].append(normal) 
            else: 
                break
        else:
            break
    path['points'] =  np.array(path['points'])
    path['normals'] =  np.array(path['normals'])
    assert path['points'].shape == path['normals'].shape, path
    return path

def gather_all_paths(stl_file, latitudes_num):
    latitudes = make_latitudes(latitudes_num)
    bridge = Bridge(stl_file)  
    face_id, point, normal = get_start(bridge.mesh)
    #print('gather_all_paths', point, normal, point.shape, normal.shape)
    paths = []
    for latitude in latitudes:
        #print(latitude)
        path = gather_path(latitude, face_id, point, normal, bridge)
        paths.append(path)
    return paths

def cumulative_distances(path):
    cumulative = [0]
    for i in range(0, len(path)-1):        
        dist = np.linalg.norm(path[i] - path[i+1])
        cumulative.append(dist+cumulative[-1])
    cumulative = np.array(cumulative)
    return cumulative

def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)
    
def sample_path(path, samples_num):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    step = 1. / samples_num
    position = np.arange(0, 1, step) + step
    samples = {'points': [], 'normals': []}
    start = 0
    for pos in position:
        pnt, nrm, start = get_sample(pos, normed[start:], path['points'][start:],
                               path['normals'][start:])
        samples['points'].append(pnt)
        samples['normals'].append(nrm)
        
    samples['points'] =  np.array(samples['points'])
    samples['normals'] =  np.array(samples['normals'])
    assert samples['points'].shape == samples['normals'].shape,  \
        "Points {} /== Normals {}".format(samples['points'].shape, samples['normals'].shape)
    return samples

def sample_all_paths(paths, samples_num):
    all_samples = []
    for path in paths:
        path_samples =  sample_path(path, samples_num)        
        #print(path_samples['points'].shape, path_samples['normals'].shape)
        all_samples.append(path_samples)
        
    #for p in  all_samples:
    #    print(p['points'].shape, p['normals'].shape)
    res_points = np.array([p['points'] for p in all_samples])
    res_normals = np.array([p['normals'] for p in all_samples])

    assert res_points.shape ==  res_normals.shape
    n, h = len(paths) // 2, len(res_points) // 2
    # # Concatenate opposite angles 0-180, 10:190, ...170:350
    return (
        np.concatenate((np.flip(res_points[h:], axis=1), res_points[:h]), axis=1),
        np.concatenate((np.flip(res_normals[h:], axis=1), res_normals[:h]), axis=1)
    )


file_name = './data/centered_3.stl'

s_no =  32
latitudes_num = s_no * 2
paths = gather_all_paths(file_name, latitudes_num)
print(len(paths))

samples_num = s_no // 2
samples = sample_all_paths(paths, samples_num)
vertices, normals = samples

64


In [3]:
latitudes_num = 4
make_latitudes(latitudes_num)

array([[[ 0.,  0.],
        [ 2.,  0.]],

       [[ 0.,  0.],
        [-0.,  2.]],

       [[ 0.,  0.],
        [-2., -0.]],

       [[ 0.,  0.],
        [ 0., -2.]]], dtype=float32)

In [4]:
# 3333333
# 3222223
# 3211123
# 3210123
# 3211123
# 3222223
# 3333333


# 0 - 1
# 1 - 6
# 2 - 
def create_layer(n):
    return 4

In [5]:
# 0 - 2 + 0*2 =  2
# 1 - 2 + 1*2 =  4
# 2 - 2 + 2*2 =  6
# 3 - 2 + 3*2 =  8
# 5 - 2 + 4*2 = 10
from math import pi


def get_angles(n):
    d = pi / 8
    p0, p1, p2, p3 = [d*(2*x+1) for x in range(4)]
    return p0, p1, p2, p3
    
p0, p1, p2, p3 = get_angles(1)    
p0, p1, p2, p3

(0.39269908169872414,
 1.1780972450961724,
 1.9634954084936207,
 2.748893571891069)

In [6]:
def get_angles_no(n):
    return 4 * (1 + n)

get_angles_no(1)

8

In [7]:
# 1111
#  00
#  00
#
def get_angles_series(n):
    return [x for x in range(0, n*2+1, 2)]
get_angles_series(4)

[0, 2, 4, 6, 8]

In [8]:
for no_angles in get_angles_series(4):
    print(get_angles_no(no_angles))

4
12
20
28
36


In [9]:
stl_file = './data/centered_3.stl'
bridge = Bridge(stl_file)  

face_id, point, normal = get_start(bridge.mesh)
print(face_id, point, normal)
latitudes_num = 4
latitudes = make_latitudes(latitudes_num)
print(latitudes)
paths = []
for latitude in latitudes:
    #print(latitude)
    path = gather_path(latitude, face_id, point, normal, bridge)
    paths.append(path)
paths    

5206 [0.     0.     0.7935] [0 0 1]
[[[ 0.  0.]
  [ 2.  0.]]

 [[ 0.  0.]
  [-0.  2.]]

 [[ 0.  0.]
  [-2. -0.]]

 [[ 0.  0.]
  [ 0. -2.]]]


[{'points': array([[ 0.    ,  0.    ,  0.7935],
         [ 0.0051, -0.0055,  0.7966],
         [ 0.0054, -0.0055,  0.7965],
         [ 0.0108, -0.0054,  0.7958],
         [ 0.0112, -0.0054,  0.7957],
         [ 0.0163, -0.0053,  0.7943],
         [ 0.0168, -0.0053,  0.7942],
         [ 0.0217, -0.0055,  0.792 ],
         [ 0.0221, -0.0058,  0.792 ],
         [ 0.0268, -0.0062,  0.7889],
         [ 0.024 ,  0.    ,  0.7868],
         [ 0.0215,  0.0064,  0.7845],
         [ 0.0261,  0.0058,  0.7815],
         [ 0.0267,  0.0051,  0.7813],
         [ 0.0305,  0.0043,  0.7781],
         [ 0.0316,  0.0034,  0.7774],
         [ 0.0346,  0.0024,  0.7743],
         [ 0.0362,  0.0015,  0.7728],
         [ 0.0386,  0.0004,  0.7702],
         [ 0.0406, -0.0005,  0.7679],
         [ 0.0424, -0.0015,  0.766 ],
         [ 0.0449, -0.0024,  0.7627],
         [ 0.0461, -0.0034,  0.7615],
         [ 0.049 , -0.0043,  0.7573],
         [ 0.0498, -0.0054,  0.7569],
         [ 0.0531, -0.0064,  0.7516],
  

In [10]:
def get_sample(pos, normed, path_points, path_normals):    
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)

def single_path_sample(path, layer_no, total_layers):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    pos = layer_no / total_layers    
    pnt, nrm, _ = get_sample(pos, normed, 
                             path['points'], path['normals'])
    return pnt, nrm
    

path, samples_num = paths[0], 4
#sample_path(path, samples_num)
layer_no, total_layers = 1, 4
pnt, nrm = single_path_sample(path, layer_no, total_layers)
pnt, nrm

(array([0.1712, 0.005 , 0.6018]), array([0.4314, 0.1092, 0.8954]))

In [11]:
path

{'points': array([[ 0.    ,  0.    ,  0.7935],
        [ 0.0051, -0.0055,  0.7966],
        [ 0.0054, -0.0055,  0.7965],
        [ 0.0108, -0.0054,  0.7958],
        [ 0.0112, -0.0054,  0.7957],
        [ 0.0163, -0.0053,  0.7943],
        [ 0.0168, -0.0053,  0.7942],
        [ 0.0217, -0.0055,  0.792 ],
        [ 0.0221, -0.0058,  0.792 ],
        [ 0.0268, -0.0062,  0.7889],
        [ 0.024 ,  0.    ,  0.7868],
        [ 0.0215,  0.0064,  0.7845],
        [ 0.0261,  0.0058,  0.7815],
        [ 0.0267,  0.0051,  0.7813],
        [ 0.0305,  0.0043,  0.7781],
        [ 0.0316,  0.0034,  0.7774],
        [ 0.0346,  0.0024,  0.7743],
        [ 0.0362,  0.0015,  0.7728],
        [ 0.0386,  0.0004,  0.7702],
        [ 0.0406, -0.0005,  0.7679],
        [ 0.0424, -0.0015,  0.766 ],
        [ 0.0449, -0.0024,  0.7627],
        [ 0.0461, -0.0034,  0.7615],
        [ 0.049 , -0.0043,  0.7573],
        [ 0.0498, -0.0054,  0.7569],
        [ 0.0531, -0.0064,  0.7516],
        [ 0.052 , -0.0005,  

In [12]:
grid = torch.zeros(4, 4)
grid

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [17]:
# 0,0 0,1 02
def square_indices(n):
    max_idx = n
    res = []
    res += [[0, x] for x in range(max_idx -1, -1, -1)]
    res += [[x, 0] for x in range(1, max_idx)]
    res += [[max_idx -1, x] for x in range(1, max_idx)]
    res += [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

n = 4
sid = square_indices(n)
sid

[[0, 3],
 [0, 2],
 [0, 1],
 [0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [3, 1],
 [3, 2],
 [3, 3],
 [2, 3],
 [1, 3]]

In [18]:
for no, (i, k) in enumerate(sid):
    grid[i, k] = no
grid  

tensor([[ 3.,  2.,  1.,  0.],
        [ 4.,  0.,  0., 11.],
        [ 5.,  0.,  0., 10.],
        [ 6.,  7.,  8.,  9.]])

In [31]:
def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res

n = 2
sid = square_indices(n)
sid

[[0, 5],
 [0, 4],
 [0, 3],
 [0, 2],
 [0, 1],
 [0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 0],
 [5, 0],
 [5, 1],
 [5, 2],
 [5, 3],
 [5, 4],
 [5, 5],
 [4, 5],
 [3, 5],
 [2, 5],
 [1, 5]]

In [20]:
grid = torch.zeros(2, 2)
print(grid)
for no, (i, k) in enumerate(sid):
    grid[i, k] = no
grid  

tensor([[0., 0.],
        [0., 0.]])


tensor([[1., 0.],
        [2., 3.]])

In [28]:
grid = torch.zeros(4, 4)
n = 1
for no, (i, k) in enumerate(sid):
    grid[i+n, k+n] = no
grid  

tensor([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 2., 3., 0.],
        [0., 0., 0., 0.]])

In [30]:
grid = torch.zeros(6, 6)
n = 2
for no, (i, k) in enumerate(sid):
    grid[i+n, k+n] = no
grid  

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 2., 3., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [42]:
def square_indices(n):
    max_idx = 2 * (n+1)
    res = [[0, x] for x in range(max_idx -1, -1, -1)]\
        + [[x, 0] for x in range(1, max_idx)]\
        + [[max_idx -1, x] for x in range(1, max_idx)]\
        + [[x, max_idx -1] for x in range(max_idx -2, 0, -1)]
    return res


square_no = 2
sid = square_indices(square_no)

n = 2
grid_side = 2*(n+1)
grid = torch.zeros(grid_side, grid_side)
offset = n-square_no
for no, (i, k) in enumerate(sid):
    grid[i+offset, k+offset] = no
grid

tensor([[ 5.,  4.,  3.,  2.,  1.,  0.],
        [ 6.,  0.,  0.,  0.,  0., 19.],
        [ 7.,  0.,  0.,  0.,  0., 18.],
        [ 8.,  0.,  0.,  0.,  0., 17.],
        [ 9.,  0.,  0.,  0.,  0., 16.],
        [10., 11., 12., 13., 14., 15.]])

In [45]:
square_no = 3
sid = square_indices(square_no)

n = 4
grid_side = 2*(n+1)
grid = torch.zeros(grid_side, grid_side)
offset = n-square_no
for no, (i, k) in enumerate(sid):
    grid[i+offset, k+offset] = no
grid  

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  0.],
        [ 0.,  8.,  0.,  0.,  0.,  0.,  0.,  0., 27.,  0.],
        [ 0.,  9.,  0.,  0.,  0.,  0.,  0.,  0., 26.,  0.],
        [ 0., 10.,  0.,  0.,  0.,  0.,  0.,  0., 25.,  0.],
        [ 0., 11.,  0.,  0.,  0.,  0.,  0.,  0., 24.,  0.],
        [ 0., 12.,  0.,  0.,  0.,  0.,  0.,  0., 23.,  0.],
        [ 0., 13.,  0.,  0.,  0.,  0.,  0.,  0., 22.,  0.],
        [ 0., 14., 15., 16., 17., 18., 19., 20., 21.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])